<a href="https://colab.research.google.com/github/0xM4sk/pub/blob/main/Mistral_7b_instruct_feature_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Note: Responses from local models can be quite slow, especially with 8-bit quantization.

With 4bit quantization, `mistralai/Mistral-7B-Instruct-v0.1` uses about 12GB of VRAM and 8.5GB of RAM. I used a T4-High RAM instance for this notebook.

In [ ]:
%pip install llama-index
%pip install transformers accelerate bitsandbytes
%pip install llama-index-readers-web
%pip install llama-index-llms-huggingface
%pip install llama-index-embeddings-huggingface
%pip install llama-index-program-openai
%pip install llama-index-agent-openai

## Setup

### Data

In [ ]:
from llama_index.readers.web import BeautifulSoupWebReader

url = "https://www.theverge.com/2023/9/29/23895675/ai-bot-social-network-openai-meta-chatbots"

documents = BeautifulSoupWebReader().load_data([url])

### LLM

This should run on a T4 instance on the free tier

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


llm = HuggingFaceLLM(
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    query_wrapper_prompt=PromptTemplate("<s>[INST] {query_str} [/INST] </s>\n"),
    context_window=3900,
    max_new_tokens=256,
    model_kwargs={"quantization_config": quantization_config},
    # tokenizer_kwargs={},
    generate_kwargs={"temperature": 0.2, "top_k": 5, "top_p": 0.95},
    device_map="auto",
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
from llama_index.core import Settings

Settings.llm = llm
Settings.embed_model = "local:BAAI/bge-small-en-v1.5"

### Index Setup

In [ ]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents)

In [ ]:
from llama_index.core.indices import SummaryIndex

summary_index = SummaryIndex.from_documents(documents)

### Helpful Imports / Logging

In [ ]:
from llama_index.core.response.notebook_utils import display_response

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

## Basic Query Engine

### Compact (default)

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="compact")

response = query_engine.query("How do OpenAI and Meta differ on AI tools?")

display_response(response)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.2` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:377: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** OpenAI and Meta differ in their approach to AI tools. OpenAI tends to present its products as productivity tools, simple utilities for getting things done. On the other hand, Meta is in the entertainment business and is building LLMs, and on Wednesday the company revealed that it has found its own uses for generative AI and voices. OpenAI's ChatGPT is a language model that can be interacted with via voice and can be used for a variety of tasks, including answering questions and providing daily affirmations. Meta, on the other hand, is building personality-driven chatbots that can be used in its messaging apps, including characters voiced by celebrities. While OpenAI's ChatGPT is focused on productivity, Meta's chatbots are focused on entertainment and social interaction.

### Refine

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="refine")

response = query_engine.query("How do OpenAI and Meta differ on AI tools?")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** OpenAI and Meta differ in their approach to AI tools. OpenAI tends to present its products as productivity tools, simple utilities for getting things done, while Meta is in the entertainment business. However, Meta is also building LLMs and has found its own uses for generative AI and voices. On the other hand, OpenAI's latest updates for ChatGPT include a voice feature that lets you interact with its large language model via voice, and the app feels much more powerful as a mobile app. The voices are earnest, upbeat, and more dynamic than what we are used to with Alexa or the Google assistant. OpenAI's ChatGPT also has a feature that lets you upload images and ask questions about them. The result is that a tool which was already useful for lots of things suddenly became useful for much more.

In addition to this, Meta is also exploring new ways to integrate AI into its social networking products. For example, the company is planning to place its AI characters on every major surface of its products, including Facebook pages and Instagram accounts. These characters will be able to be messaged in the same inbox as friends and family, and it's possible that they will also be able to make Reels

### Tree Summarize

In [ ]:
query_engine = vector_index.as_query_engine(response_mode="tree_summarize")

response = query_engine.query("How do OpenAI and Meta differ on AI tools?")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** OpenAI and Meta differ in their approach to AI tools. OpenAI tends to present its products as productivity tools, simple utilities for getting things done. On the other hand, Meta is in the entertainment business and is building LLMs, and on Wednesday the company revealed that it has found its own uses for generative AI and voices. OpenAI's ChatGPT is a language model that can be interacted with via voice and can be used for a variety of tasks, including answering questions and providing daily affirmations. Meta, on the other hand, is building personality-driven chatbots that can be used in its messaging apps, including characters voiced by celebrities. While OpenAI's ChatGPT is focused on productivity, Meta's chatbots are focused on entertainment and social networking.

## Router Query Engine

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts."
    )
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document."
    )
)

### Single Selector

In [ ]:
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    select_multi=False
)

response = query_engine.query("What was mentioned about Meta?")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


OutputParserException: ignored

### Multi Selector

In [ ]:
from llama_index.core.query_engine import RouterQueryEngine

query_engine = RouterQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    select_multi=True,
)

response = query_engine.query("What was mentioned about Meta? Summarize with any other companies mentioned in the entire document.")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** Meta was announced to be integrating AI chatbots into its messaging apps, which will be based on the personalities and likenesses of celebrities, including MrBeast, Dwyane Wade, Kendall Jenner, and Paris Hilton. The chatbots are designed for entertainment rather than productivity. The shift towards a more synthetic social network could have implications for how people engage with each other on these platforms. Other companies mentioned in the entire document include OpenAI, Taylor Swift and other celebrities like Charli D'Amelio, Dwyane Wade, Kendall Jenner, MrBeast, Snoop Dogg, Tom Brady, and Paris Hilton.

## SubQuestion Query Engine

In [ ]:
from llama_index.core.tools import QueryEngineTool, ToolMetadata

vector_tool = QueryEngineTool(
    vector_index.as_query_engine(),
    metadata=ToolMetadata(
        name="vector_search",
        description="Useful for searching for specific facts."
    )
)

summary_tool = QueryEngineTool(
    summary_index.as_query_engine(response_mode="tree_summarize"),
    metadata=ToolMetadata(
        name="summary",
        description="Useful for summarizing an entire document."
    )
)

In [ ]:
import nest_asyncio
nest_asyncio.apply()

In [ ]:
from llama_index.core.query_engine import SubQuestionQueryEngine

query_engine = SubQuestionQueryEngine.from_defaults(
    [vector_tool, summary_tool],
    verbose=True,
)

response = query_engine.query("What was mentioned about Meta? How Does it differ from how OpenAI is talked about?")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Generated 4 sub questions.
[vector_search] Q: Vector search for 'Meta'


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[vector_search] A: 
Based on the provided context information, a vector search for "Meta" would reveal the following:

* Meta, the company formerly known as Facebook, is actively researching and developing artificial intelligence (AI) technologies.
* Meta has announced a new feature for its messaging apps that uses AI-generated imagery and personality-driven chatbots based on the voices of celebrities.
* Meta has also announced the release of 28 chatbot characters based on the voices of popular celebrities.
* The chatbot feature is designed to be a productivity tool that simplifies various tasks, while the chatbot characters are meant to entertain users in various ways.
* The technology is new, and the company is using this feature as a testing ground before fully incorporating AI chatbots in its social networking platforms.
* The technology has the potential to revolutionize social networking by creating a partially synthetic social network that could be more engaging, personalized, a

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[vector_search] A: 
This article, titled "The synthetic social network is coming", discusses the latest updates to OpenAI's chatbot service, which now includes voice functionality. This feature allows users to interact with the chatbot using speech, giving the bot a more human-like personality. The article also talks about Meta's AI characters, which are being developed by the company. These characters are based on the personalities of celebrities and are used in Meta's messaging apps. The article suggests that the use of AI in social networking is evolving and that this technology is likely to become more prominent in the future.
[summary] Q: Summary of OpenAI mentions


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[summary] A: 
OpenAI, a leading company in the development of Artificial Intelligence (AI), recently announced new updates and features for its language model, ChatGPT. These updates include a voice feature that allows users to interact with the app via voice, as well as the ability to upload images and ask questions about them. These updates make the app much more powerful and engaging, with users experiencing the app in a more natural and human way. The voice feature gives ChatGPT a hint of personality, making it seem more empathetic, patient and available to its users. This is the first step in a new era of consumer internet, where AI-powered chatbots and companions become more prevalent and accessible to users, providing emotional support and companionship to those who may be lonely, isolated, or on the margins. Meanwhile, OpenAI's competitors such as Meta, are also developing AI chatbots and voices, with the company launching 28 personality-driven chatbots to be used in its messag

**`Final Response:`** Meta is mentioned as a technology company that is actively researching and developing artificial intelligence (AI) technologies. The company recently announced a new feature for its messaging apps that uses AI-generated imagery and personality-driven chatbots based on the voices of celebrities. The chatbot

## SQL Query Engine

Here, we download and use a sample SQLite database with 11 tables, with various info about music, playlists, and customers. We will limit to a select few tables for this test.

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!curl https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip -O /content/chinook.zip
!unzip /content/chinook.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  298k  100  298k    0     0  1000k      0 --:--:-- --:--:-- --:--:-- 1001k
curl: (3) URL using bad/illegal format or missing URL
Archive:  /content/chinook.zip
  inflating: chinook.db              


In [ ]:
from sqlalchemy import create_engine, MetaData, Table, Column, String, Integer, select, column

engine = create_engine("sqlite:////content/chinook.db")

In [ ]:
from llama_index.core import SQLDatabase

sql_database = SQLDatabase(engine)

In [ ]:
from llama_index.core.indices.struct_store import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["albums", "tracks", "artists"],
)

In [ ]:
response = query_engine.query("What are some albums? Limit it to 5.")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** Here are five popular albums that might interest you: 
1. War by Pink Floyd 
2. Lost, Season 2 by The Lumineers 
3. Sir Neville Marriner: A Celebration by Sir Neville Marriner 
4. Out Of Exile by Drake 
5. Instant Karma: The Amnesty International Campaign to Save Darfur by The Police

In [ ]:
response = query_engine.query("What are some artists? Limit it to 5.")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


**`Final Response:`** 1. Aerosmith
2. AC/DC
3. Alice In Chains
4. Anthrax
5. Black Sabbath

This last query should be a more complex join

In [ ]:
response = query_engine.query("What are some tracks from the artist AC/DC? Limit it to 3")

display_response(response)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


NotImplementedError: ignored

In [ ]:
print(response.metadata['sql_query'])

## Programs

Depending the LLM, you will have to test with either `OpenAIPydanticProgram` or `LLMTextCompletionProgram`

In [ ]:
from typing import List
from pydantic import BaseModel

from llama_index.core.program import LLMTextCompletionProgram
from llama_index.program.openai import OpenAIPydanticProgram

class Song(BaseModel):
    """Data model for a song."""

    title: str
    length_seconds: int


class Album(BaseModel):
    """Data model for an album."""

    name: str
    artist: str
    songs: List[Song]

In [ ]:
from llama_index.core.output_parsers import PydanticOutputParser

prompt_template_str = """\
Generate an example album, with an artist and a list of songs. \
Using the movie {movie_name} as inspiration.\
"""
program = LLMTextCompletionProgram.from_defaults(
    output_parser=PydanticOutputParser(Album),
    prompt_template_str=prompt_template_str,
    llm=llm,
    verbose=True,
)

In [ ]:
output = program(movie_name="The Shining")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ValidationError: ignored

In [ ]:
print(output)

## Data Agent

Similar to programs, OpenAI LLMs will use `OpenAIAgent`, while other LLMs will use `ReActAgent`.

In [ ]:
from llama_index.core.agent import ReActAgent
from llama_index.agent.openai import OpenAIAgent

agent = ReActAgent.from_tools(
    [vector_tool, summary_tool],
    llm=llm,
    verbose=True
)

It seems tool usage is pretty flakey

In [ ]:
response = agent.chat("Hello!")
print(response)

In [ ]:
response = agent.chat("What was mentioned about Meta? How Does it differ from how OpenAI is talked about?")
print(response)